# Stable deffusion Google Colab Edition

---


## 1. モデルの読み込み Load Checkpoint

In [1]:
import torch
from diffusers import StableDiffusionXLPipeline, DPMSolverMultistepScheduler, EulerAncestralDiscreteScheduler
from transformers import CLIPTokenizer, CLIPTextModel
from IPython.display import Image, display_png
from PIL import Image
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/MODEL/pelucheMIXColourfulTartX_teddybearV1.safetensors"  # @param {type:"string"}

scheduler_name = "Euler A"  # @param ["Euler A", "DPM Solver++"]

# モデルをロード
pipe = StableDiffusionXLPipeline.from_single_file(
    model_path,
    torch_dtype=torch.float16
).to("cuda")

# スケジューラを適用
if scheduler_name == "DPM Solver++":
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
elif scheduler_name == "Euler A":
    pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
else:
    raise ValueError("無効なスケジューラー名です")

## 2. LoRaの読み込み Load LoRa

In [ ]:
lora_dict = {
    "/content/drive/MyDrive/MODEL/748cmSDXL.safetensors": 0.4,
    "/content/drive/MyDrive/MODEL/Detailed anime style.safetensors": 0.5,
}  # LoRaを読み込むにはここを変更する

# LoRaの読み込み
try:
    lora_names = []
    for idx, (lora_path, scale) in enumerate(lora_dict.items()):
        lora_name = f"lora_{idx}"
        pipe.load_lora_weights(lora_path, adapter_name=lora_name) # 読み込む
        lora_names.append(lora_name)
        print(f"LoRa {lora_path} が 重み {scale} で読み込まれました")

    pipe.set_adapters(lora_names, list(lora_dict.values()))  # LoRaの比率を適用
    print("すべてのLoRaが正しく読み込まれました!")
except Exception as e:
    print(f"LoRaの読み込みに失敗しました: {e}")

##3.  画像生成 Image generation

prompt :プロンプト

negative_prompt :ネガティブプロンプト

image_width, image_height :画像の大きさ  大きいと時間が掛かる

num_steps :サンプリング回数  多いと時間が掛かる

seed :シード  数値以外を入力でランダム

scale :CFGスケール

count :枚数

In [ ]:
import torch
import shutil
import os
from PIL import Image
from IPython.display import display

# パラメータ設定（Colab 用）
prompt = "1girl, long hair, blunt bangs, blue eyes, school, whole body, thin lines, anime style, masterpiece, score_9, score_8_up"  # @param {"type":"string","placeholder":"プロンプトを入力"}
negative_prompt = "(long torso), halo, bad quality, error, worst quality, worst detail, bad, low quality, bad anatomy, text, 6 fingers, bad hands, bad feet, bad hand, bad finger, bad fingers, extra finger, extra fingers, split finger, split fingers, extra digits, fused arms, fused hands"  # @param {"type":"string","placeholder":"ネガティブプロンプトを入力"}
image_width = 768  # @param {"type":"number","placeholder":"横幅"}
image_height = 1024  # @param {"type":"number","placeholder":"縦幅"}
num_steps = 20  # @param {"type":"number","placeholder":"サンプリング回数"}
seed = None  # @param {"type":"number","placeholder":"シード値 (空白でランダム)"}
scale = 4  # @param {"type":"number","placeholder":"CFG scale"}
count = 8  # @param {"type":"number","placeholder":"生成する枚数"}

# 出力フォルダとZIP保存先
output_dir = "/content/generated_images"
zip_path = "/content/generated_images.zip"

# 出力フォルダを作成（既に存在していれば削除して作り直す）
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# 画像生成ループ
for i in range(count):
    seed_value = int(seed) if isinstance(seed, int) else torch.randint(0, 2**32, (1,)).item()
    generator = torch.manual_seed(seed_value)

    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        width=image_width,
        height=image_height,
        num_inference_steps=num_steps,
        guidance_scale=scale,
        generator=generator
    ).images[0]

    # 画像の保存
    image_path = os.path.join(output_dir, f"{i}.png")
    image.save(image_path)
    print(f"{i + 1} / {count} 枚目 シード: {seed_value}")

    # 画像を表示（Colab 用）
    display(Image.open(image_path))

# 画像を ZIP 圧縮
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', output_dir)

print(f"画像を {zip_path} に保存しました。")